# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
import pandas as pd
import numpy as np
import sklearn 
from sqlalchemy import create_engine
from nltk.tokenize import word_tokenize 
from nltk.stem import WordNetLemmatizer
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.ensemble import RandomForestClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import classification_report
import re
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, make_scorer
from sklearn.model_selection import train_test_split, GridSearchCV
import pickle

In [2]:
import nltk
nltk.download(['punkt', 'wordnet'])

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [3]:
# load data from database
engine = create_engine('sqlite:///disaster_response.db')
df=pd.read_sql_table('messages',engine)

In [4]:
X = df.message
Y = df.iloc[:,4:]

In [5]:
print(df)

          id                                            message  \
0          2  Weather update - a cold front from Cuba that c...   
1          7            Is the Hurricane over or is it not over   
2          8                    Looking for someone but no name   
3          9  UN reports Leogane 80-90 destroyed. Only Hospi...   
4         12  says: west side of Haiti, rest of the country ...   
5         14             Information about the National Palace-   
6         15                     Storm at sacred heart of jesus   
7         16  Please, we need tents and water. We are in Sil...   
8         17    I would like to receive the messages, thank you   
9         18  I am in Croix-des-Bouquets. We have health iss...   
10        20  There's nothing to eat and water, we starving ...   
11        21  I am in Petionville. I need more information r...   
12        22  I am in Thomassin number 32, in the area named...   
13        24  Let's do it together, need food in Delma 75, i..

### 2. Write a tokenization function to process your text data

In [15]:
def tokenize(text):
    url_regex = 'http[s]?://(?:[a-z-AZ]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'
    detected_urls = re.findall(url_regex, text)
    for url in detected_urls:
        text = text.replace(url, "urlplaceholder")

    tokens = word_tokenize(text)
    lemmatizer = WordNetLemmatizer()

    clean_tokens = []
    for tok in tokens:
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_tok)

    return clean_tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [16]:
pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer = tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(RandomForestClassifier()))
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

X_train, Y_train, X_test, Y_Test = train_test_split(X, Y, test_size = 0.3, random_state=(42))

pipeline.fit(X_train, Y_train)

In [17]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, random_state = 1)

np.random.seed(17)
pipeline.fit(X_train, Y_train)

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [18]:
Y_pred = pipeline.predict(X_test)

In [19]:
for n, column in enumerate(Y):
    print(column, classification_report(Y_test.iloc[:,n],Y_pred[:,n]))

related              precision    recall  f1-score   support

          0       0.62      0.33      0.43      1550
          1       0.81      0.94      0.87      4951
          2       0.53      0.17      0.26        53

avg / total       0.77      0.79      0.76      6554

request              precision    recall  f1-score   support

          0       0.88      0.98      0.93      5415
          1       0.83      0.38      0.53      1139

avg / total       0.87      0.88      0.86      6554

offer              precision    recall  f1-score   support

          0       1.00      1.00      1.00      6528
          1       0.00      0.00      0.00        26

avg / total       0.99      1.00      0.99      6554

aid_related              precision    recall  f1-score   support

          0       0.71      0.88      0.79      3815
          1       0.75      0.51      0.61      2739

avg / total       0.73      0.72      0.71      6554

medical_help              precision    recall  f1-sco

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### 6. Improve your model
Use grid search to find better parameters. 

In [ ]:
parameters = {
    'clf__estimator__n_estimators':[50,100,200],
    'clf__estimator__min_samples_split':[2,3,3,4]
}

cv =  GridSearchCV(pipeline,parameters)
cv.fit(X_train, Y_train)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [ ]:
Y_pred = pipeline.predict(X_test)

In [ ]:
for n, column in enumerate(Y):
    print(column, classification_report(Y_test.iloc[:,n],Y_pred[:,n]))

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 9. Export your model as a pickle file

In [ ]:
pickle.dump(tuned_model, open('disaster_model.sav', 'wb'))

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.